In [1]:
import matplotlib
matplotlib.use(qt)
# import lmfit as lm
import numpy as np 
# import utils as u
# import EDXS_model as em
import hyperspy.api as hs
import matplotlib.pyplot as plt 
from pathlib import Path
from snmfem.conf import DB_PATH
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Span

output_notebook()

ImportError: Cannot load backend 'Qt5Agg' which requires the 'qt5' interactive framework, as 'headless' is currently running

In [ ]:
import matplotlib
print(matplotlib.get_backend())

# Data loading

In [ ]:
filename = DB_PATH / Path("71GPa_subsolidus.hspy")
spectrum = hs.load(filename)

if len(spectrum.axes_manager.shape) == 3 :
    y = spectrum.data.mean(axis =(0,1))
else : 
    y = spectrum.data

x=np.linspace(spectrum.axes_manager[0].offset,
spectrum.axes_manager[0].offset+spectrum.axes_manager[0].size*spectrum.axes_manager[0].scale,num=spectrum.axes_manager[0].size)

# Fitting regions

## Choosing the fitting regions

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook

output_notebook()

# output_file("toolbar.html")
pan = Span(location = 3, dimension = "height")
# create a new plot with the toolbar below
p = figure(plot_width=400, plot_height=400,
           title=None, toolbar_location="below",
           toolbar_sticky=False)
p.add_layout(pan)

p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)

show(p)

In [ ]:
spectrum.plot()
roi1, roi2, roi3, roi4, roi5 =hs.roi.SpanROI(1,2), hs.roi.SpanROI(3,4), hs.roi.SpanROI(5,6), hs.roi.SpanROI(7,8), hs.roi.SpanROI(9,10)
span1, span2, span3, span4, span5 = roi1.interactive(spectrum), roi2.interactive(spectrum), roi3.interactive(spectrum), roi4.interactive(spectrum), roi5.interactive(spectrum)

## Saving the fitting regions

In [ ]:
list_energies=[[roi1.left,roi1.right],[roi2.left,roi2.right],[roi3.left,roi3.right],[roi4.left,roi4.right],[roi5.left,roi5.right]]

In [ ]:
list_energies =[[0.6476500000000001, 0.75265], [3.1526500000000004, 3.8676500000000003], [5.6526499999999995, 6.127649999999999], [11.46765, 12.18265], [13.12265, 15.332650000000001]]

# Fitting the Data

## Segmenting the data

In [ ]:
part_x=np.linspace(list_energies[0][0],list_energies[0][1],num=round((list_energies[0][1]-list_energies[0][0])/spectrum.axes_manager[0].scale))
part_y=spectrum.isig[list_energies[0][0]:list_energies[0][1]].data
for elt in list_energies[1:] :
    part_x=np.append(part_x,np.linspace(elt[0],elt[1],num=round((elt[1]-elt[0])/spectrum.axes_manager[0].scale)))
    part_y=np.append(part_y,spectrum.isig[elt[0]:elt[1]].data)

list_boola=[]
for i in range(len(list_energies)) :
    list_boola.append(np.empty_like(x))

for i in range(len(list_energies)) :
    for j in range(x.shape[0]) :
        if np.logical_and(x[j]<list_energies[i][1],x[j]>list_energies[i][0]) :
            list_boola[i][j]=True
        else :
            list_boola[i][j] = False

sum_boola=np.zeros_like(x,dtype=bool)
for elt in list_boola :
    sum_boola+=elt.astype(bool)

## Fitting model

In [ ]:
from scipy.special import erfc

def simple_abs_coeff (x) :
    return 1/np.power(x,3)

def brstlg (x,b0,b1,b2) :
    return b0/x + b1 + b2*x

def detector(x,c1,c2) :
    return np.exp(-c2/np.power(x,3))*(1-np.exp(-c1/np.power(x,3)))

def self_abs(x,c0) :
    return (1-np.exp(-c0/np.power(x,3)))/(c0/np.power(x,3))

def shelf (x,height,length) :
    return height*erfc(x-length)

In [ ]:
model = em.EDXS_Model("Data/simple_xrays_threshold.json","Data/wernisch_abs.json")
model.x = part_x
model.generate_abs_coeff({"8":1.0,"12" : 0.51,"14":0.61,"13":0.07,"20":0.04,"62":0.02,"26":0.028,"60":0.002,"71":0.003,"72":0.003,"29" : 0.02})

abs = model.abs
def brstlg (x,b0,b1,b2) :
    return b0/x + b1 + b2*x

def detector(x,c1,c2) :
    return np.exp(-c2*abs)*(1-np.exp(-c1*abs))

def self_abs(x,c0) :
    return (1-np.exp(-c0*abs))/(c0*abs)



## Setting initial parameters values

In [ ]:
Amod = lm.Model(self_abs)
Bmod = lm.Model(brstlg)
Dmod = lm.Model(detector)
Smod = lm.Model(shelf)

full_mod = Amod * Bmod * Dmod #+ Smod
pars = full_mod.make_params()
pars["c0"].set(value=1.0,min=0)
pars["c1"].set(value=1000.0,min=0)
pars["c2"].set(value = 0.01, min=0)
pars["b0"].set(value = 0.006)
pars["b1"].set(value = -3.6e-4)
pars["b2"].set(value = 4.1e-5,min=0)
# pars["height"].set(value=0.000671381	,min=0)
# pars["length"].set(value = 0.69193039,min = 0 )

## Fitting Procedure

In [ ]:
results = full_mod.fit(part_y,x=part_x,params=pars)

## Plotting the results with the data

In [ ]:
plt.plot(x,y,"ko",markersize=3,label="exp")
plt.fill_between(x,0,y,where=sum_boola,label="fit windows",color="0.8")
plt.xlim(0.03,20)
plt.ylim(0,3)
plt.xlabel("Energy (keV)",fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel("Intensity",fontsize=22)
plt.plot(x,results.model.eval(results.params,x=x),"r-",linewidth=3,label="fit")
plt.legend(fontsize=22)

## Plotting the components of the model

In [ ]:
plt.plot(x,Amod.eval(results.params,x=x),"-.",label="self abs")
plt.plot(x,Bmod.eval(results.params,x=x),label="bremsstrahlung")
plt.plot(x,Dmod.eval(results.params,x=x),"--",label="detector")
#plt.plot(x,Smod.eval(results.params,x=x),".",label="shelf")
#plt.plot(x,y,"ko",label="data")
plt.xlabel("Energy (keV)",fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel("Intensity",fontsize=22)
plt.legend(fontsize=22)
plt.ylim(0,1.1)
plt.xlim(0.01,20)

In [ ]:
results.params